In [ ]:
import os
import re
import sys
import json
import glob
import datetime
import rasterio
import numpy as np
from matplotlib import pyplot as plt

import cv2
import fiona
import rasterio.mask

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

gdalenv_bin = '/home/keith/anaconda3/envs/gdalenv/bin/'

### Load a Landsat project

In [ ]:
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    reset=False,    # whether to delete everything and start from scratch
    refresh=False   # whether to re-run all cached operations
)

### Future features

In [ ]:
# search for operations
operations = proj.find_operations(method='stack', bands=[4,3,2])

# show portions of the operations tree
proj.show_descendents(operation)
proj.show_ancestors(operation)

# open the dataset produced by an operation
im = proj.open_dataset(operation='last')
im = proj.open_dataset(operation=ind)

# re-run selected or all operations
proj.rerun_operations(operations='all')

### Load a DEM project

In [ ]:
# transform colormap file into dict
with open('../../../_projects-db/sierra-map/data/cmap/cmap_crop2.txt', 'r') as file:
    cmap = [[int(val) for val in line.strip().split(' ')] for line in file.readlines()]
cmap = [{'elevation': row[0]*3.28, 'color': tuple(np.array(row[1:]))} for row in cmap]

In [ ]:
cmap

In [ ]:
# DEM without resetting
proj = managers.DEMProject(
    project_root='../projects/test-dem', 
    dataset_paths='../test/datasets/dem/n38w120/n38w120_100m.tif',
    reset=False)

In [ ]:
path = proj.operations[2].destination.path
with rasterio.open(path) as src:
    im = src.read()
    profile = src.profile

In [ ]:
proj.operations

### Rasterizing a shapefile

### Roads from OSM

These shapefiles downloaded from http://download.geofabrik.de/north-america.html

In [ ]:
info = managers.utils.shell(
    'fio info /home/keith/Downloads/norcal-latest-free.shp/gis_osm_roads_free_1.shp', 
    verbose=False)
json.loads(info)

In [ ]:
# generate roads-all.json
d = {}
features = []
road_types = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary']

with fiona.open('/home/keith/Downloads/norcal-latest-free.shp/gis_osm_roads_free_1.shp', "r") as shapefile:
    for ind, feature in shapefile.items():
        if feature['properties']['fclass'] in road_types:
            features.append(feature)
        
with open('/home/keith/Downloads/norcal-latest-free.shp/roads-all.json', 'w') as file:
    d['features'] = features
    json.dump(d, file)

In [ ]:
# geometry for just one road
road_startswith = 'Hauser Bridge'
with fiona.open('/home/keith/Downloads/norcal-latest-free.shp/gis_osm_roads_free_1.shp', "r") as shapefile:
    d = {}
    features = []
    for ind, feature in shapefile.items():
        if feature['properties']['name'] and feature['properties']['name'].startswith(road_startswith):
            features.append(feature)

with open('/home/keith/Downloads/norcal-latest-free.shp/roads-one.json', 'w') as file:
    d['features'] = features
    json.dump(d, file)

### Rasterize `roads-all.json` using `rasterio.rasterize`

In [ ]:
from affine import Affine
from rasterio import plot as rplot
from rasterio.crs import CRS
from rasterio.features import rasterize
from rasterio.features import bounds as calculate_bounds

In [ ]:
with open('/home/keith/Downloads/norcal-latest-free.shp/roads-all.json') as file:
    geojson = json.load(file)

geometries = [feature['geometry'] for feature in geojson['features']]
len(geometries)

# bounds = geojson.get('bbox', calculate_bounds(geojson))
bounds = (-124, 38, -122, 40)
res = (.002, .002)

width = max(int(np.ceil((bounds[2] - bounds[0]) / float(res[0]))), 1)
height = max(int(np.ceil((bounds[3] - bounds[1]) / float(res[1]))), 1)

src_crs = CRS.from_string('EPSG:4326')

kwargs = {
    'count': 1,
    'crs': src_crs,
    'width': width,
    'height': height,
    'transform': Affine(res[0], 0, bounds[0], 0, -res[1], bounds[3]),
    'driver': 'GTiff'
}

result = rasterize(
    geometries,
    out_shape=(kwargs['height'], kwargs['width']),
    transform=kwargs['transform'],
    dtype=kwargs.get('dtype'),
    all_touched=True,
    default_value=255,
    fill=0)

In [ ]:
result_last = result.copy()

In [ ]:
roads = result.astype(float)/255
roads_last = result_last.astype(float)/255

In [ ]:
rplot.show(result, transform=kwargs['transform'])

In [ ]:
plt.figure(figsize=(12,12))
final = (255*(roads + roads_last/2)).astype('uint8')
plt.imshow(final)

In [ ]:
np.unique(final[:])

In [ ]:
kwargs['dtype'] = 'uint8'
kwargs['driver'] = 'GTiff'
kwargs['tiled'] = False

with rasterio.open('test.tif', 'w', **kwargs) as out:
    out.write(final, indexes=1)

### Skipping the rio CLI by calling the underlying methods directly

Calling the underlying methods requires commenting out all of the decorators in `rio/<command>.py`, which will break the actual rio CLI. Testing this out on `rio warp`. 

In [ ]:
from rasterio.rio import warp

In [ ]:
# mock the click package's context object
class Context:
    pass

verbosity = 0
ctx = Context()
ctx.obj = {}
ctx.obj["verbosity"] = verbosity
envopts = {"CPL_DEBUG": (verbosity > 2)}
ctx.obj["env"] = rasterio.Env(**envopts)

In [ ]:
files = ['../projects/test-dem/test-dem_hillshade_20190110-215946.TIF']
output = '../projects/test-dem/test_warp.tif'

warp.warp(ctx, files, output, 'GTiff', None, 'EPSG:4269', 
          None, None, None, [], 'nearest', 
          None, None, 1, True, True, {'tiled': False}, False)